## Create weather animation with GEOS-R data

In [2]:

urls2dwn = [ 'https://noaa-goes16.s3.amazonaws.com/ABI-L1b-RadM/2022/306/17/OR_ABI-L1b-RadM1-M6C03_G16_s20223061734250_e20223061734308_c20223061734345.nc',
  'https://noaa-goes16.s3.amazonaws.com/ABI-L1b-RadM/2022/306/17/OR_ABI-L1b-RadM1-M6C03_G16_s20223061735250_e20223061735308_c20223061735348.nc',
  'https://noaa-goes16.s3.amazonaws.com/ABI-L1b-RadM/2022/306/17/OR_ABI-L1b-RadM1-M6C03_G16_s20223061737250_e20223061737308_c20223061737344.nc',
  'https://noaa-goes16.s3.amazonaws.com/ABI-L1b-RadM/2022/306/17/OR_ABI-L1b-RadM1-M6C03_G16_s20223061738250_e20223061738308_c20223061738344.nc',
  'https://noaa-goes16.s3.amazonaws.com/ABI-L1b-RadM/2022/306/17/OR_ABI-L1b-RadM1-M6C03_G16_s20223061739250_e20223061739308_c20223061739344.nc',
]

In [10]:
import requests
import os

# Iterate through the URLs and download files.
for urld in urls2dwn:
    # Extract the filename from the URL.
    ntw = urld.split('/')[-1]
    
    ### new name directory to create  ---> Jorge 
    ndir = ntw.split('_')[-3]
    ### new directory path            ---> Jorge
    local_directory = f"Output_data/ABI-L1b-RadM/{ndir}"
    ###                               ---> Jorge
    os.makedirs(local_directory, exist_ok=True)
    
    
    # Construct the complete path to save the file in the local directory.
    file_path = os.path.join(local_directory, ntw)
    
    # Send an HTTP GET request to the URL.
    resp = requests.get(urld)
    
    # Check if the response is successful (status code 200).
    if resp.status_code == 200:
        # Write the content to the file in binary mode.
        with open(file_path, "wb") as file:
            file.write(resp.content)
        print(f"File '{ntw}' downloaded and saved to '{local_directory}'.")
    else:
        print(f"Failed to download '{ntw}' from the URL: {urld}")

File 'OR_ABI-L1b-RadM1-M6C03_G16_s20223061734250_e20223061734308_c20223061734345.nc' downloaded and saved to 'Output_data/ABI-L1b-RadM/s20223061734250'.
File 'OR_ABI-L1b-RadM1-M6C03_G16_s20223061735250_e20223061735308_c20223061735348.nc' downloaded and saved to 'Output_data/ABI-L1b-RadM/s20223061735250'.
File 'OR_ABI-L1b-RadM1-M6C03_G16_s20223061737250_e20223061737308_c20223061737344.nc' downloaded and saved to 'Output_data/ABI-L1b-RadM/s20223061737250'.
File 'OR_ABI-L1b-RadM1-M6C03_G16_s20223061738250_e20223061738308_c20223061738344.nc' downloaded and saved to 'Output_data/ABI-L1b-RadM/s20223061738250'.
File 'OR_ABI-L1b-RadM1-M6C03_G16_s20223061739250_e20223061739308_c20223061739344.nc' downloaded and saved to 'Output_data/ABI-L1b-RadM/s20223061739250'.


In [1]:
# Importing necessary libraries for data manipulation, visualization, and animation creation.

# numpy is used for handling numerical operations on arrays. It provides support for large,
# multi-dimensional arrays and matrices, along with a large collection of high-level mathematical
# functions to operate on these arrays. Essential for processing numerical data efficiently.
import numpy as np

# matplotlib.pyplot is a plotting library used for creating static, interactive, and animated visualizations in Python.
# It provides an object-oriented API for embedding plots into applications.
import matplotlib.pyplot as plt

# matplotlib.animation is used to create animations using matplotlib. This module provides a framework for creating
# animated plots and updating them over time, which is crucial for creating dynamic visualizations of weather patterns.
import matplotlib.animation as animation

# IPython.display.HTML is used to display HTML documents in the Jupyter notebook. It will be used to display the animation
# inline within the Jupyter notebook, making it easily accessible and interactive.
from IPython.display import HTML

# glob is used for file handling. It provides a function for making file lists from directory wildcard searches,
# which simplifies the process of finding and managing paths to multiple data files, common in satellite data storage.
from glob import glob

# xarray is specifically designed to work with labeled data and multidimensional arrays. It excels at handling
# netCDF files which are commonly used for storing multi-dimensional meteorological data, making it ideal for 
# accessing and manipulating satellite data like that from GEOS-R.
import xarray as xr


In [2]:
# Import necessary libraries for file and path operations
import os
from glob import glob

# Set up the directory path dynamically. This method allows the script to be run on any machine without modification,
# regardless of the operating system, by adapting to the user's current working directory.

# Obtain the current working directory where this script or notebook is being executed.
current_directory = os.getcwd()

# Append the specific subdirectory 'ABI-L1b-RadM1' to the current working directory. 
# This is the directory where the GEOS-R satellite data files, specifically ABI L1b Radiance files, are expected to be stored.
full_directory_path = os.path.join(current_directory, 'ABI-L1b-RadM1')

# Use the glob function to create a list of file paths that match a specific pattern within the directory.
# The pattern '**/OR_ABI-L1b-RadM1-M6C03*.nc' is designed to match any files that start with 'OR_ABI-L1b-RadM1-M6C03'
# and end with '.nc', indicating NetCDF files of a particular channel (Channel 3 in this case) from the dataset.
# The '**' allows for matching this pattern at any level of depth within the specified directory, facilitating the inclusion of all
# subdirectories in the search.
glst = glob(f'{full_directory_path}/**/OR_ABI-L1b-RadM1-M6C03*.nc', recursive=True)

# Display the list of file paths to verify that the correct files are being found.
# This step is crucial for debugging and ensuring that the data setup is correct before proceeding to data processing or analysis.
glst

['/media/ismart/jhbravo/po/Module_4/ABI-L1b-RadM1/s20223061734250/OR_ABI-L1b-RadM1-M6C03_G16_s20223061734250_e20223061734308_c20223061734345.nc',
 '/media/ismart/jhbravo/po/Module_4/ABI-L1b-RadM1/s20223061735250/OR_ABI-L1b-RadM1-M6C03_G16_s20223061735250_e20223061735308_c20223061735348.nc',
 '/media/ismart/jhbravo/po/Module_4/ABI-L1b-RadM1/s20223061737250/OR_ABI-L1b-RadM1-M6C03_G16_s20223061737250_e20223061737308_c20223061737344.nc',
 '/media/ismart/jhbravo/po/Module_4/ABI-L1b-RadM1/s20223061738250/OR_ABI-L1b-RadM1-M6C03_G16_s20223061738250_e20223061738308_c20223061738344.nc',
 '/media/ismart/jhbravo/po/Module_4/ABI-L1b-RadM1/s20223061739250/OR_ABI-L1b-RadM1-M6C03_G16_s20223061739250_e20223061739308_c20223061739344.nc']

In [3]:
# Import necessary libraries for handling arrays, plotting, and animations.
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import xarray as xr

# Create a new figure for plotting. This figure will serve as the canvas for our animation.
fig = plt.figure()

# Initialize a list to store frames for the animation. Each frame will consist of an image plot of satellite data.
ims = []

# Loop through each file path in the list 'glst', which contains paths to NetCDF files with satellite imagery.
for fl in glst:
    # Open the dataset from the current file using xarray. Xarray enables easy handling of multidimensional
    # scientific data and seamlessly integrates with plotting libraries for visualization.
    ncx = xr.open_dataset(fl)

    # Create an image from the 'Rad' (Radiance) data in the dataset. This involves:
    # - Accessing the 'Rad' data array from the dataset.
    # - Plotting this data array using matplotlib's imshow function, which is ideal for displaying images on a 2D grid.
    # - Setting the colormap to 'Greys_r' to display data in grayscale, which helps in emphasizing intensity variations.
    # - Marking the image as 'animated' so it can be managed by matplotlib's animation tools.
    im = plt.imshow(ncx['Rad'].data, origin='upper', cmap="Greys_r", animated=True)
    
    # Append the created image plot to the list of frames. Each frame is a list itself, containing the image plot.
    ims.append([im])

# Close the plt.show() window to prevent display of a static plot, ensuring that only the animation is displayed.
plt.close()

# Create an animation object from the list of image frames. This uses matplotlib's ArtistAnimation,
# which compiles the frames into a sequence. Parameters include:
# - 'fig', the figure object to animate.
# - 'ims', the list of frames to include in the animation.
# - 'interval=50', setting the display duration of each frame to 50 milliseconds.
# - 'blit=True', enabling blitting, which improves animation performance by only redrawing parts of the frame that have changed.
# - 'repeat_delay=1000', setting a 1000 millisecond delay before the animation repeats.
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)

# Convert the animation to a JavaScript HTML representation using IPython.display.HTML. This conversion allows
# the animation to be embedded and interactively played within a Jupyter Notebook. It's an effective way to
# visualize dynamic processes like weather patterns in satellite data.
HTML(ani.to_jshtml())